In [2]:
import pandas as pd
import os
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report, f1_score, roc_auc_score
import pickle
import Functions2
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report

In [3]:
# Load the data
data_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv"
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

In [4]:
# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(Functions2.measure_pigment_network(original))
            feature_2.append(Functions2.measure_blue_veil(original))
            feature_3.append(Functions2.measure_vascular(original))
            feature_4.append(Functions2.measure_globules(original))
            feature_5.append(Functions2.measure_streaks(original))
            feature_6.append(Functions2.measure_irregular_pigmentation(original))
            feature_7.append(Functions2.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

In [5]:
# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Training"

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

In [6]:
# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Merge the features DataFrame with the diagnostic column from the original DataFrame
df_merged = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

In [7]:
# Split the data into training and testing sets
X = df_merged.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

In [8]:
# Logistic Regression
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, Y_train)
logreg_prediction = logreg_classifier.predict(X_test)

# Evaluate Logistic Regression
logreg_cm = confusion_matrix(Y_test, logreg_prediction)
logreg_precision = precision_score(Y_test, logreg_prediction)
logreg_recall = recall_score(Y_test, logreg_prediction)
logreg_f1 = f1_score(Y_test, logreg_prediction)
logreg_auc_roc = roc_auc_score(Y_test, logreg_prediction)
logreg_classification_rep = classification_report(Y_test, logreg_prediction)

# Print Logistic Regression evaluation metrics
print("Logistic Regression:")
print("Confusion Matrix:")
print(logreg_cm)
print("Precision:", logreg_precision)
print("Recall:", logreg_recall)
print("F1 Score:", logreg_f1)
print("AUC-ROC Score:", logreg_auc_roc)
print("Classification Report:")
print(logreg_classification_rep)

Logistic Regression:
Confusion Matrix:
[[15  6]
 [ 6 17]]
Precision: 0.7391304347826086
Recall: 0.7391304347826086
F1 Score: 0.7391304347826085
AUC-ROC Score: 0.7267080745341614
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.71      0.71        21
           1       0.74      0.74      0.74        23

    accuracy                           0.73        44
   macro avg       0.73      0.73      0.73        44
weighted avg       0.73      0.73      0.73        44



In [9]:
# Save LR model
with open(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\Pickle\logreg_model.pkl', 'wb') as file:
    pickle.dump(logreg_classifier, file)

# Save KNN model
with open(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\Pickle\knn_model.pkl', 'wb') as file:
    pickle.dump(knn_classifier, file)

NameError: name 'knn_classifier' is not defined

In [10]:
# Predict the diagnostic for the test set
test_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"
test_feature_1, test_feature_2, test_feature_3, test_feature_4, test_feature_5, test_feature_6, test_feature_7 = extract_features(test_path)
test_features_df = pd.DataFrame()
test_features_df["img_id"] = [filename for filename in os.listdir(test_path) if filename.endswith(('.jpg', '.png'))]
test_features_df["1: pigment network"] = test_feature_1
test_features_df["2: Blue veil"] = test_feature_2
test_features_df["3: Vascular"] = test_feature_3
test_features_df["4: Globules"] = test_feature_4
test_features_df["5: Streaks"] = test_feature_5
test_features_df["6: Pigmentation"] = test_feature_6
test_features_df["7: Regression"] = test_feature_7

# Load the model
with open(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\Pickle\logreg_model.pkl', 'rb') as file:
    logreg_model = pickle.load(file)

# Predict the diagnostic for the test set
test_X = test_features_df.drop(['img_id'], axis=1)
test_Y = logreg_model.predict(test_X)

# Create a DataFrame for the predictions
test_predictions_df = pd.DataFrame()
test_predictions_df["img_id"] = test_features_df["img_id"]
test_predictions_df["diagnostic"] = test_Y

# Print the predictions
print(test_predictions_df)

                  img_id  diagnostic
0    PAT_109_868_723.png           1
1    PAT_236_361_180.png           0
2      PAT_26_37_865.png           0
3    PAT_270_417_257.png           1
4   PAT_333_1469_499.png           0
5      PAT_38_54_234.png           1
6    PAT_398_799_163.png           0
7       PAT_42_58_13.png           1
8      PAT_56_86_479.png           1
9      PAT_59_46_537.png           0
10   PAT_76_1039_269.png           0
11    PAT_998_17_641.png           0


In [11]:
# Find the mean of the features for each diagnostic
df_mean = df_merged.groupby('diagnostic').mean()
print(df_mean)

            1: pigment network  2: Blue veil  3: Vascular  4: Globules  \
diagnostic                                                               
0                     9.484829   5065.066667  2642.711111    59.022222   
1                     8.262954   6085.093023  3373.953488    83.813953   

            5: Streaks  6: Pigmentation  7: Regression  
diagnostic                                              
0             1.247442        13.459710     695.400000  
1             1.273240        11.654273     374.906977  


In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance

# Assuming you have your data in X and y variables
# X should be a 2D array-like object with shape (n_samples, n_features)
# y should be a 1D array-like object with shape (n_samples,)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

# Initialize and train a logistic regression classifier
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Perform permutation importance for logistic regression
result_lr = permutation_importance(lr, X_test, y_test, n_repeats=10, random_state=42)

# Get the feature importance scores for logistic regression
importance_scores_lr = result_lr.importances_mean

# Sort the feature indices based on importance scores in descending order for logistic regression
sorted_indices_lr = np.argsort(importance_scores_lr)[::-1]

# Initialize and train a k-nearest neighbors classifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Perform permutation importance for k-nearest neighbors
result_knn = permutation_importance(knn, X_test, y_test, n_repeats=10, random_state=42)

# Get the feature importance scores for k-nearest neighbors
importance_scores_knn = result_knn.importances_mean

# Sort the feature indices based on importance scores in descending order for k-nearest neighbors
sorted_indices_knn = np.argsort(importance_scores_knn)[::-1]

# Print the sorted features and their importance scores for logistic regression
print("Feature Importance Scores for Logistic Regression:")
for i in sorted_indices_lr:
    print(f"Feature {i+1}: {importance_scores_lr[i]}")

# Print the sorted features and their importance scores for k-nearest neighbors
print("Feature Importance Scores for K-Nearest Neighbors:")
for i in sorted_indices_knn:
    print(f"Feature {i+1}: {importance_scores_knn[i]}")


Feature Importance Scores for Logistic Regression:
Feature 5: 2.2204460492503132e-17
Feature 7: 0.0
Feature 4: -0.004545454545454508
Feature 2: -0.011363636363636343
Feature 1: -0.020454545454545402
Feature 6: -0.020454545454545427
Feature 3: -0.027272727272727226
Feature Importance Scores for K-Nearest Neighbors:
Feature 2: 0.12954545454545446
Feature 7: 0.0022727272727272483
Feature 6: 0.0
Feature 5: 0.0
Feature 4: 0.0
Feature 1: 0.0
Feature 3: -0.013636363636363691


In [16]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.datasets import make_classification

# Generate some example data (replace this with your own dataset)
X, y = make_classification(n_samples=100, n_features=20, random_state=42)

# Create a SelectKBest object with the desired score function (f_classif for classification)
k_best = SelectKBest(score_func=f_classif, k=10)

# Perform feature selection
X_selected = k_best.fit_transform(X, y)

# Get the selected feature indices
selected_indices = k_best.get_support(indices=True)

# Print the selected feature indices
print("Selected feature indices:", selected_indices)


Selected feature indices: [ 1  3  4  5  6  7  8 10 15 17]
